# PINECONEを用いたvectorサーチのデモ

llama_indexのバージョンが上がったので、それに伴いコードを修正

2. 技術ブログの登録

- https://gpt-index.readthedocs.io/en/stable/how_to/integrations/vector_stores.html


# 方針

- LangChain.Agentを利用せずに、LLamaIndex単体で実装

In [8]:
import os

from dotenv import load_dotenv
from llama_index.vector_stores import PineconeVectorStore
from llama_index import SimpleWebPageReader, LLMPredictor, ServiceContext, OpenAIEmbedding, GPTVectorStoreIndex, StorageContext
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
import pinecone
from langchain.chat_models import ChatOpenAI

INDEX_NAME = "chatgpt-search-index"

PREDICTOR_MODEL_NAME = "gpt-3.5-turbo"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

INITIAL_URLS = [
    "https://dev.classmethod.jp/articles/lang-chain-agent-customized-by-llama-index-tool/",
    "https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155",
    "https://runble1.com/gcp-terraform-cloud-run/"
]
ADDITIONAL_URL = "https://zenn.dev/tfutada/articles/acf8adbb2ba5be"

# カスタムテンプレートの作成
CUSTOM_TEXT_QA_PROMPT_TMPL = (
    "コンテキストは以下です. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "コンテキストが与えられた場合, "
    "質問に回答してください: {query_str}\n"
)
CUSTOM_TEXT_QA_PROMPT = QuestionAnswerPrompt(CUSTOM_TEXT_QA_PROMPT_TMPL)

CUSTOM_REFINE_PROMPT_TMPL = (
    "元の質問: {query_str}\n"
    "オリジナルの回答: {existing_answer}\n"
    "以下のコンテキストを使って、オリジナルの回答を推敲することができます.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "コンテキストを元に、オリジナルの回答を、より元の質問に沿ったものに推敲してください. "
    "もしコンテキストが有用なものでなければ、オリジナルの回答を返却してください"
)
CUSTOM_REFINE_PROMPT = RefinePrompt(CUSTOM_REFINE_PROMPT_TMPL)

# 環境変数の読み込み
load_dotenv('../.env')

True

In [3]:
# デバッグする際に実行
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# 既存DBへの接続 & 検索

In [5]:
# clientの作成
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENVIRONMENT"])
pinecone_index = pinecone.Index(INDEX_NAME)

vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index
)

In [6]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [12]:
# indexのセットアップ
storage_context = StorageContext.from_defaults(
    vector_store = vector_store
)

llm_predictor = LLMPredictor(
    llm=ChatOpenAI(
        temperature=0, model_name=PREDICTOR_MODEL_NAME, openai_api_key=os.environ["OPENAI_API_KEY"]
    )
)

embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL_NAME
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)

index = GPTVectorStoreIndex.from_documents(
    documents=[], service_context=service_context, storage_context=storage_context
)

Unknown max input size for gpt-3.5-turbo, using defaults.
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [13]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [15]:
query_engine = index.as_query_engine(
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT,
    similarity_top_k=3
)

search_query = "calibrationってどんな技術だっけ？"

response = query_engine.query(
    search_query
)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["calibration\\u3063\\u3066\\u3069\\u3093\\u306a\\u6280\\u8853\\u3060\\u3063\\u3051\\uff1f"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["calibration\\u3063\\u3066\\u3069\\u3093\\u306a\\u6280\\u8853\\u3060\\u3063\\u3051\\uff1f"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=199 request_id=b167086ef5724a8b5451d7f148ea030b response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=199 request_id=b167086ef5724a8b5451d7f148ea030b response_code=200
DEBUG:llama_index.indices.utils:> Top 3 nodes:
> [Node 

api_version=None data='{"messages": [{"role": "user", "content": "\\u30b3\\u30f3\\u30c6\\u30ad\\u30b9\\u30c8\\u306f\\u4ee5\\u4e0b\\u3067\\u3059. \\n---------------------\\nurl: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155\\n\\nurl: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155\\n\\n[ \\u8aad\\u8005\\u306b\\u306a\\u308b\\n](https://blog.hatena.ne.jp/YukoIshizaki/yukoishizaki.hatenablog.com/subscribe?utm_campaign=subscribe_blog&utm_source=blogs_topright_button&utm_medium=button)\\n\\n# [\\u6a5f\\u68b0\\u5b66\\u7fd2 Memo \\u03c6(\\u30fb\\u03c9\\u30fb )](https://yukoishizaki.hatenablog.com/)\\n\\n\\u3053\\u306e\\u5e83\\u544a\\u306f\\u300190\\u65e5\\u4ee5\\u4e0a\\u66f4\\u65b0\\u3057\\u3066\\u3044\\u306a\\u3044\\u30d6\\u30ed\\u30b0\\u306b\\u8868\\u793a\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002\\n\\n[ 2020-05-24 ](https://yukoishizaki.hatenablog.com/archive/2020/05/24)\\n\\n#\\n[\\u78ba\\u7387\\u4e88\\u6e2c\\u3068Calibration\\u306b\\u3064\\u3044\\u3066](https://yu

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=9262 request_id=8a7a2faead5491445c10a41a9dc4741a response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=9262 request_id=8a7a2faead5491445c10a41a9dc4741a response_code=200
DEBUG:llama_index.llm_predictor.base:Calibrationは、確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。
Calibrationは、確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。
DEBUG:llama_index.indices.response.response_builder:> Initial prompt template: コンテキストは以下です. 
---------------------
url: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155

url: https://yukoishizaki.hatenablog.com/entry/2020/05/24/14515

DEBUG:llama_index.indices.response.response_builder:> Initial response: Calibrationは、確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。
> Initial response: Calibrationは、確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。
DEBUG:llama_index.indices.response.response_builder:> Refine context: set algorithms for isotonic regression; A
unify...
> Refine context: set algorithms for isotonic regression; A
unify...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: calibration\\u3063\\u306

api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: calibration\\u3063\\u3066\\u3069\\u3093\\u306a\\u6280\\u8853\\u3060\\u3063\\u3051\\uff1f\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: Calibration\\u306f\\u3001\\u78ba\\u7387\\u4e88\\u6e2c\\u306e\\u4fe1\\u983c\\u6027\\u3092\\u9ad8\\u3081\\u308b\\u305f\\u3081\\u306e\\u6280\\u8853\\u3067\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u51fa\\u529b\\u5024\\u3092\\u5404\\u30af\\u30e9\\u30b9\\u306b\\u5c5e\\u3059\\u308b\\u78ba\\u7387\\u306b\\u8fd1\\u3065\\u3051\\u308b\\u3053\\u3068\\u3092\\u6307\\u3057\\u307e\\u3059\\u3002\\u5177\\u4f53\\u7684\\u306b\\u306f\\u3001Sigmoid / Platt Scale\\u3084Isotonic Regression\\u306a\\u3069\\u306e\\u624b\\u6cd5\\u3092\\u4f7f\\u3063\\u3066\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u51fa\\u529b\\u5024\\u3092\\u6b63\\u89e3\\u30e9\\u30d9\\u30eb\\u306e\\u30af\\u30e9\\u30b9\\u5206\\u5e03\\u306b\\u8fd1\\u3065\\u3051\\u307e\\u3059\\u3002\\u307e\\u305f\\u3001Calibration Curve\

...

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=12146 request_id=79357fd1ef4a882e8ffbe86f42cd2741 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=12146 request_id=79357fd1ef4a882e8ffbe86f42cd2741 response_code=200
DEBUG:llama_index.llm_predictor.base:Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。
Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。
DEBUG:llama_index.indices.response.response_builder:> Refined prompt template: 元の質問: calibrationってどんな技術だっけ？
オリジナルの回答: Calibrationは、モデルの確率予測の信頼性を高め

DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: calibration\\u3063\\u3066\\u3069\\u3093\\u306a\\u6280\\u8853\\u3060\\u3063\\u3051\\uff1f\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: Calibration\\u306f\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u78ba\\u7387\\u4e88\\u6e2c\\u306e\\u4fe1\\u983c\\u6027\\u3092\\u9ad8\\u3081\\u308b\\u305f\\u3081\\u306e\\u6280\\u8853\\u3067\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u51fa\\u529b\\u5024\\u3092\\u5404\\u30af\\u30e9\\u30b9\\u306b\\u5c5e\\u3059\\u308b\\u78ba\\u7387\\u306b\\u8fd1\\u3065\\u3051\\u308b\\u3053\\u3068\\u3092\\u6307\\u3057\\u307e\\u3059\\u3002\\u5177\\u4f53\\u7684\\u306b\\u306f\\u3001Sigmoid / Platt Scale\\u3084Isotonic Regression\\u306a\\u3069\\u306e\\u624b\\u6cd5\\u3092\\u4f7f\\u3063\\u3066\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u51fa\\u529b\\u5024\\u3092\\u6b63\\u89e3\\u30e9\\u30d9\\u30eb\\u306e\\u30af\\u30e9\\u30b9\\u5206\\u5e03\\u306b\\u8fd1\\u3065\\u3051\\u307e\\u3059\\u30

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=19038 request_id=9bdfa2630b1fc0bfbe2498267b77427f response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=19038 request_id=9bdfa2630b1fc0bfbe2498267b77427f response_code=200
DEBUG:llama_index.llm_predictor.base:Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。また、MMMなどの統計分析だけでは過学習している可能性があるため、交差検証を行うことが重要です。A/Bテストを行うことで、交差検証を補完し、モデルの信頼性を高めることができます。実験は、努力を積み重ねて出来るようにしておくべきものです。
Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。また、MMMなどの統計分析だけでは過学習している可能性があるため、交差検証を行うことが重要です。A/Bテストを行うことで、交差検証を補完し、モデルの信頼性を高めることができます。実験は、努力を積み重ねて出来るようにしておくべきものです。
DEBU

api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: calibration\\u3063\\u3066\\u3069\\u3093\\u306a\\u6280\\u8853\\u3060\\u3063\\u3051\\uff1f\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: Calibration\\u306f\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u78ba\\u7387\\u4e88\\u6e2c\\u306e\\u4fe1\\u983c\\u6027\\u3092\\u9ad8\\u3081\\u308b\\u305f\\u3081\\u306e\\u6280\\u8853\\u3067\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u51fa\\u529b\\u5024\\u3092\\u5404\\u30af\\u30e9\\u30b9\\u306b\\u5c5e\\u3059\\u308b\\u78ba\\u7387\\u306b\\u8fd1\\u3065\\u3051\\u308b\\u3053\\u3068\\u3092\\u6307\\u3057\\u307e\\u3059\\u3002\\u5177\\u4f53\\u7684\\u306b\\u306f\\u3001Sigmoid / Platt Scale\\u3084Isotonic Regression\\u306a\\u3069\\u306e\\u624b\\u6cd5\\u3092\\u4f7f\\u3063\\u3066\\u3001\\u30e2\\u30c7\\u30eb\\u306e\\u51fa\\u529b\\u5024\\u3092\\u6b63\\u89e3\\u30e9\\u30d9\\u30eb\\u306e\\u30af\\u30e9\\u30b9\\u5206\\u5e03\\u306b\\u8fd1\\u3065\\u3051\\u307e\\u3059\\u3002\\u6700\\u8

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=16752 request_id=32e94a4e404bfdc1483a14ce16a237f8 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=16752 request_id=32e94a4e404bfdc1483a14ce16a237f8 response_code=200
DEBUG:llama_index.llm_predictor.base:Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。また、MMMなどの統計分析だけでは過学習している可能性があるため、交差検証を行うことが重要です。A/Bテストを行うことで、交差検証を補完し、モデルの信頼性を高めることができます。
Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。また、MMMなどの統計分析だけでは過学習している可能性があるため、交差検証を行うことが重要です。A/Bテストを行うことで、交差検証を補完し、モデルの信頼性を高めることができます。
DEBUG:llama_index.indices.response.response_builder:> Refined 

In [16]:
print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

response: Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。また、MMMなどの統計分析だけでは過学習している可能性があるため、交差検証を行うことが重要です。A/Bテストを行うことで、交差検証を補完し、モデルの信頼性を高めることができます。
参照url1: https://tjo.hatenablog.com/entry/2023/04/26/191100
参照url2: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155


/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/llama_index/data_structs/node.py:181: UserWarning: .extra_info is deprecated, use .node.extra_info instead
  warnings.warn(".extra_info is deprecated, use .node.extra_info instead")


In [17]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

# データのインサート

In [18]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [20]:
search_query = "DIDについて教えて"

response = query_engine.query(
    search_query
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=183 request_id=d6e454fddf13a9455ef1ed0f9bd39f21 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=183 request_id=d6e454fddf13a9455ef1ed0f9bd39f21 response_code=200
DEBUG:llama_index.indices.utils:> Top 3 nodes:
> [Node 49d15913-0249-4a56-bc4d-2f14b477a42e] [Similarity score:             0.4

api_version=None data='{"messages": [{"role": "user", "content": "\\u30b3\\u30f3\\u30c6\\u30ad\\u30b9\\u30c8\\u306f\\u4ee5\\u4e0b\\u3067\\u3059. \\n---------------------\\nurl: https://qiita.com/nokoxxx1212/items/544dff3bd39dd1773638\\n\\nurl: https://qiita.com/nokoxxx1212/items/544dff3bd39dd1773638\\n\\n[](/)\\n\\nsearchsearch[Login](/login?redirectTo=%2Fnokoxxx1212%2Fitems%2F544dff3bd39dd1773638)[Signup](/signup?redirectTo=%2Fnokoxxx1212%2Fitems%2F544dff3bd39dd1773638)\\n\\n[Trend](/)[Question](/question-trend)[Official Event](/official-\\nevents)[Official Column](/official-\\ncolumns)[Opportunities](/opportunities)[Organization](/organizations)\\n\\n[\\u30d7\\u30ed\\u30b0\\u30e9\\u30df\\u30f3\\u30b0\\u8a00\\u8a9eRuby\\u306e\\u751f\\u307f\\u306e\\u89aa \\u307e\\u3064\\u3082\\u3068\\u3086\\u304d\\u3072\\u308d\\u6c0f\\u767b\\u58c7\\uff01Qiita Conference\\n2023\\u30925/17(\\u6c34)-19(\\u91d1)\\u306b\\u958b\\u50ac\\ud83d\\ude80](https://increments.connpass.com/event/279734/)close\\n\\n[3

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4200 request_id=71b8cf84221a141730827a18e5403390 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4200 request_id=71b8cf84221a141730827a18e5403390 response_code=200
DEBUG:llama_index.llm_predictor.base:提供されたコンテキストからは、DIDに関する情報は見つかりませんでした。DIDについての情報を提供していただければ、回答することができます。
提供されたコンテキストからは、DIDに関する情報は見つかりませんでした。DIDについての情報を提供していただければ、回答することができます。
DEBUG:llama_index.indices.response.response_builder:> Initial prompt template: コンテキストは以下です. 
---------------------
url: https://qiita.com/nokoxxx1212/items/544dff3bd39dd1773638

url: https://qiita.com/nokoxxx1212/items/544dff3bd39dd1773638

[](/)

searchsearch[Login](/login?redirectTo=%2Fnokoxxx1212%2Fitems%2F544dff3bd39dd1773638)[Signup](/signup?redirectTo=%2Fnokoxxx1212%2Fitems%2F544dff3bd39dd1773638)

[Trend](/)[Question](/question-trend)[Official Event](/official-
events)[Official 

DEBUG:llama_index.indices.response.response_builder:> Initial response: 提供されたコンテキストからは、DIDに関する情報は見つかりませんでした。DIDについての情報を提供していただければ、回答することができます。
> Initial response: 提供されたコンテキストからは、DIDに関する情報は見つかりませんでした。DIDについての情報を提供していただければ、回答することができます。
DEBUG:llama_index.indices.response.response_builder:> Refine context:  __（参考）ディレクトリ構成案

minimal-module

    
    
   ...
> Refine context:  __（参考）ディレクトリ構成案

minimal-module

    
    
   ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: \\u63d0\\u4f9b\\u3055\\u308c\\u305f\\u30b3\\u30f3\\u30c6\\u30ad\\u30b9\\u30c8\\u304b\\u3089\\u306f\\u3001DID\\u306b\\u95a2\\u3059\\u308b\\u60c5\\u5831\\u306f

api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: \\u63d0\\u4f9b\\u3055\\u308c\\u305f\\u30b3\\u30f3\\u30c6\\u30ad\\u30b9\\u30c8\\u304b\\u3089\\u306f\\u3001DID\\u306b\\u95a2\\u3059\\u308b\\u60c5\\u5831\\u306f\\u898b\\u3064\\u304b\\u308a\\u307e\\u305b\\u3093\\u3067\\u3057\\u305f\\u3002DID\\u306b\\u3064\\u3044\\u3066\\u306e\\u60c5\\u5831\\u3092\\u63d0\\u4f9b\\u3057\\u3066\\u3044\\u305f\\u3060\\u3051\\u308c\\u3070\\u3001\\u56de\\u7b54\\u3059\\u308b\\u3053\\u3068\\u304c\\u3067\\u304d\\u307e\\u3059\\u3002\\n\\u4ee5\\u4e0b\\u306e\\u30b3\\u30f3\\u30c6\\u30ad\\u30b9\\u30c8\\u3092\\u4f7f\\u3063\\u3066\\u3001\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54\\u3092\\u63a8\\u6572\\u3059\\u308b\\u3053\\u3068\\u304c\\u3067\\u304d\\u307e\\u3059.\\n------------\\n __\\uff08\\u53c2\\u8003\\uff09\\u30c7\\u30a3\\u30ec\\u30af\\u30c8\\u30ea\\u6

...

DEBUG:llama_index.indices.response.response_builder:> Refined response: DIDとは、Decentralized Identifierの略で、分散型識別子を意味します。従来の識別子は中央集権的な管理に依存していましたが、DIDはブロックチェーン技術を利用して分散型で管理されます。DIDは、個人や組織が自分自身を識別するために使用され、プライバシー保護やデータの所有権の確立に役立ちます。DIDは、W3Cが策定した標準規格であり、現在、多くの企業や団体がDIDに関する研究や開発を進めています。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。また、DIDは、個人や組織が自分自身を識別するために使用されるため、データの所有権の確立にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。
> Refined response: DIDとは、Decentralized Identifierの略で、分散型識別子を意味します。従来の識別子は中央集権的な管理に依存していましたが、DIDはブロックチェーン技術を利用して分散型で管理されます。DIDは、個人や組織が自分自身を識別するために使用され、プライバシー保護やデータの所有権の確立に役立ちます。DIDは、W3Cが策定した標準規格であり、現在、多くの企業や団体がDIDに関する研究や開発を進めています。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。また、DIDは、個人や組織が自分自身を識別するために使用されるため、データの所有権の確立にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=16600 request_id=77d5b0075d80d08e3c4599d4bbb1e5fd response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=16600 request_id=77d5b0075d80d08e3c4599d4bbb1e5fd response_code=200
DEBUG:llama_index.llm_predictor.base:DIDは、個人や組織が自分自身を識別するために使用され、プライバシー保護やデータの所有権の確立に役立つ分散型識別子です。これは、働き方やデータ活用支援、福利厚生、データ分析などの分野で重要な役割を果たすことができます。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。
DIDは、個人や組織が自分自身を識別するために使用され、プライバシー保護やデータの所有権の確立に役立つ分散型識別子です。これは、働き方やデータ活用支援、福利厚生、データ分析などの分野で重要な役割を果たすことができます。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。
DEBUG:llama_index.indices.response.response_builder:> Refined prompt template: 元の質問: DIDについて教えて
オリジナルの回答: DIDとは、Decentralized Identifierの略で、分散型識別子を意味します。従来の識別子は中央集権的な管理に依存し

api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: DID\\u306f\\u3001\\u500b\\u4eba\\u3084\\u7d44\\u7e54\\u304c\\u81ea\\u5206\\u81ea\\u8eab\\u3092\\u8b58\\u5225\\u3059\\u308b\\u305f\\u3081\\u306b\\u4f7f\\u7528\\u3055\\u308c\\u3001\\u30d7\\u30e9\\u30a4\\u30d0\\u30b7\\u30fc\\u4fdd\\u8b77\\u3084\\u30c7\\u30fc\\u30bf\\u306e\\u6240\\u6709\\u6a29\\u306e\\u78ba\\u7acb\\u306b\\u5f79\\u7acb\\u3064\\u5206\\u6563\\u578b\\u8b58\\u5225\\u5b50\\u3067\\u3059\\u3002\\u3053\\u308c\\u306f\\u3001\\u50cd\\u304d\\u65b9\\u3084\\u30c7\\u30fc\\u30bf\\u6d3b\\u7528\\u652f\\u63f4\\u3001\\u798f\\u5229\\u539a\\u751f\\u3001\\u30c7\\u30fc\\u30bf\\u5206\\u6790\\u306a\\u3069\\u306e\\u5206\\u91ce\\u3067\\u91cd\\u8981\\u306a\\u5f79\\u5272\\u3092\\u679c\\u305f\\u3059\\u3053\\u3068\\u304c\\u3067\\u304d\\u307e\\u3059\\u3002DID\\u306f\\u3001\\u30d6\\u30ed\\u30c3\\u30af\

DIDは、分散型識別子であり、個人や組織が自分自身を識別するために使用されます。これは、プライバシー保護やデータの所有権の確立に役立ち、データ分析や福利厚生などの分野で重要な役割を果たすことができます。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。特に、データ分析やITコンサルタント、ビジネス職などの分野で、DIDの活用が期待されています。
DEBUG:llama_index.indices.response.response_builder:> Refined prompt template: 元の質問: DIDについて教えて
オリジナルの回答: DIDは、個人や組織が自分自身を識別するために使用され、プライバシー保護やデータの所有権の確立に役立つ分散型識別子です。これは、働き方やデータ活用支援、福利厚生、データ分析などの分野で重要な役割を果たすことができます。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。
以下のコンテキストを使って、オリジナルの回答を推敲することができます.
------------
 * [ データ分析 (5) ](https://blog.brainpad.co.jp/archive/category/%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90)
  * [ プロジェクトインタビュー (5) ](https://blog.brainpad.co.jp/archive/category/%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%82%A4%E3%83%B3%E3%82%BF%E3%83%93%E3%83%A5%E3%83%BC)
  * [ ビジネス職 (4) ](https://blog.brainpad.co.jp/archive/categ

> Refined response: DIDは、分散型識別子であり、個人や組織が自分自身を識別するために使用されます。これは、プライバシー保護やデータの所有権の確立に役立ち、データ分析や福利厚生などの分野で重要な役割を果たすことができます。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。特に、データ分析やITコンサルタント、ビジネス職などの分野で、DIDの活用が期待されています。
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 15829 tokens
> [get_response] Total LLM token usage: 15829 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens
response: DIDは、分散型識別子であり、個人や組織が自分自身を識別するために使用されます。これは、プライバシー保護やデータの所有権の確立に役立ち、データ分析や福利厚生などの分野で重要な役割を果たすことができます。DIDは、ブロックチェーン技術を利用することで、中央集権的な管理に依存しないため、よりセキュアであり、プライバシー保護にも役立ちます。現在、多くの企業や団体がDIDに関する研究や開発を進めており、今後ますます注目される技術となるでしょう。特に、データ分析やITコンサルタント、ビジネス職などの分野で、DIDの活用が期待されています。
参照url1: https://blog.brainpad.co.jp/entry/2022/12/22/154432
参照url2: https://qiita.com/ik-fib/items/b4a502d173a22b3947a0
参照url3: 

In [22]:
# webページからdocumentクラスを作成
samples_urls = ["https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0"]


documents = SimpleWebPageReader(html_to_text=True).load_data(samples_urls)

1

In [23]:
# extra_infoにurlを追加
for document, url in zip(documents, samples_urls):
    document.extra_info = {"url": url}

In [24]:
for document in documents:
        index.insert(document)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: [Zenn](/)

[![Shingo Uto](https://storage.googl...
> Adding chunk: [Zenn](/)

[![Shingo Uto](https://storage.googl...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:  はじめに

差分の差分法(DID)を用いた因果推論手法について、Pythonによる実装を交え...
> Adding chunk:  はじめに

差分の差分法(DID)を用いた因果推論手法について、Pythonによる実装を交え...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: **特講を受講してもしなくても生じる差**
(前期試験と後期試験の難易度の違いなど)も含まれて...
> Adding chunk: **特講を受講してもしなくても生じる差**
(前期試験と後期試験の難易度の違いなど)も含まれて...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:  回帰分析による差分の差分法(DID)

差分の差分法(DID)は回帰分析のフレームワークに当...
> Adding chunk:  回帰分析による差分の差分法(DID)

差分の差分法(DID)は回帰分析のフレームワークに当...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 
    * 100点満点とし、平均50、標準偏差10の正規分布に従う
  * 特講受講ダミー...
> Adding chunk: 
    * 100点満点とし、平均50、標準偏差10の正規分布に従う
  * 特講受講ダミー...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:   # データフレームに格納し、5つだけ出力
    df = pd.DataFrame({"...
> Adding chunk:  

api_version=None data='{"input": ["url: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0  [Zenn](/)  [![Shingo Uto](https://storage.googleapis.com/zenn-user- upload/avatar/59143ddaff.jpeg)Shingo Uto](/s1ok69oo)  \\ud83d\\udc0d  # Python\\u3067\\u56e0\\u679c\\u63a8\\u8ad6(9)~\\u5dee\\u5206\\u306e\\u5dee\\u5206\\u6cd5(DID)\\u306b\\u3088\\u308b\\u52b9\\u679c\\u691c\\u8a3c~  2022/07/24\\u306b\\u516c\\u958b\\u30fb\\u7d049,100\\u5b57  [\\u30c4\\u30a4\\u30fc\\u30c8](https://twitter.com/intent/tweet?url=https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0&text=Python%E3%81%A7%E5%9B%A0%E6%9E%9C%E6%8E%A8%E8%AB%96\\\\(9\\\\)~%E5%B7%AE%E5%88%86%E3%81%AE%E5%B7%AE%E5%88%86%E6%B3%95\\\\(DID\\\\)%E3%81%AB%E3%82%88%E3%82%8B%E5%8A%B9%E6%9E%9C%E6%A4%9C%E8%A8%BC~%EF%BD%9CShingo%20Uto&hashtags=zenn)  [![](https://storage.googleapis.com/zenn-user- upload/topics/ebddf7c6bb.png)Python](/topics/python)[#\\u30c7\\u30fc\\u30bf\\u30b5\\u30a4\\u30a8\\u30f3\\u30b9](/topics/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=176 request_id=7d100ae63211449851230364748882b1 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=176 request_id=7d100ae63211449851230364748882b1 response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["url: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0   * \\u5ca9\\u6ce2\\u30c7\\u30fc\\u30bf\\u30b5\\u30a4\\u30a8\\u30f3\\u30b9\\u520a\\u884c\\u59d4\\u54e1\\u4f1a\\u7de8[\\u300c\\u5ca9\\u6ce2\\u30c7\\u30fc\\u30bf\\u30b5\\u30a4\\u30a8\\u30f3\\u30b9Vol.3-[\\u7279\\u96c6]\\u56e0\\u679c\\u63a8\\u8ad6-\\u5b9f\\u4e16\\u754c\\u306e\\u30c7\\u30fc\\u30bf\\u304b\\u3089\\u56e0\\u679c\\u3092\\u8aad\\u3080\\u300d](https://www.amazon.co.jp/%E5%B2%A9%E6%B3%A2

api_version=None data='{"input": ["url: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0   * \\u5ca9\\u6ce2\\u30c7\\u30fc\\u30bf\\u30b5\\u30a4\\u30a8\\u30f3\\u30b9\\u520a\\u884c\\u59d4\\u54e1\\u4f1a\\u7de8[\\u300c\\u5ca9\\u6ce2\\u30c7\\u30fc\\u30bf\\u30b5\\u30a4\\u30a8\\u30f3\\u30b9Vol.3-[\\u7279\\u96c6]\\u56e0\\u679c\\u63a8\\u8ad6-\\u5b9f\\u4e16\\u754c\\u306e\\u30c7\\u30fc\\u30bf\\u304b\\u3089\\u56e0\\u679c\\u3092\\u8aad\\u3080\\u300d](https://www.amazon.co.jp/%E5%B2%A9%E6%B3%A2%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9-Vol-3-%E5%B2%A9%E6%B3%A2%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9%E5%88%8A%E8%A1%8C%E5%A7%94%E5%93%A1%E4%BC%9A/dp/4000298534/ref=sr_1_1?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=VXL98DX04ZRT&keywords=%E5%B2%A9%E6%B3%A2%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9+%E5%9B%A0%E6%9E%9C%E6%8E%A8%E8%AB%96&qid=1659852526&s=books&sprefix=%E5%B2%A9%E6%B3%A2%E3%83%87%E3%83%BC%E3%8

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=200 request_id=b111059e1c817a53596a11214a144301 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=200 request_id=b111059e1c817a53596a11214a144301 response_code=200
INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 13199 tokens
> [insert] Total embedding token usage: 13199 tokens


In [25]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 62}},
 'total_vector_count': 62}

In [26]:
search_query = "DIDについて教えて"

response = query_engine.query(
    search_query
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=223 request_id=27f3fc8029b0f61fdcfa0741edee212e response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=223 request_id=27f3fc8029b0f61fdcfa0741edee212e response_code=200
DEBUG:llama_index.indices.utils:> Top 3 nodes:
> [Node 5d3d6a3a-c935-409f-a24e-5c29ba122517] [Similarity score:             0.4

api_version=None data='{"messages": [{"role": "user", "content": "\\u30b3\\u30f3\\u30c6\\u30ad\\u30b9\\u30c8\\u306f\\u4ee5\\u4e0b\\u3067\\u3059. \\n---------------------\\nurl: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0\\n\\nurl: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0\\n\\n * [\\u50be\\u5411\\u30b9\\u30b3\\u30a2\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c](https://zenn.dev/s1ok69oo/articles/c058108acb83e7)\\n  * [\\u64cd\\u4f5c\\u5909\\u6570\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c](https://zenn.dev/s1ok69oo/articles/3b75bbfd12b6a2)\\n  * [\\u56de\\u5e30\\u4e0d\\u9023\\u7d9a\\u30c7\\u30b6\\u30a4\\u30f3\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c](https://zenn.dev/s1ok69oo/articles/f40ca93fe31a8d)\\n  * [\\u6a5f\\u68b0\\u5b66\\u7fd2\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c](https://zenn.dev/s1ok69oo/articles/1eeebe75842a50)\\n\\n\\u307e\\u305f\\u3001\\u904e\\u53bb\\u306bLT\\u3084\\u52c9\\u5f37\\u4f1a\\u3067\\u767a\\

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=13385 request_id=bc5f588e0da1c08eb52399b78b3fd1d0 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=13385 request_id=bc5f588e0da1c08eb52399b78b3fd1d0 response_code=200
DEBUG:llama_index.llm_predictor.base:DIDとは、差分の差分法 (Difference-in-Differences) のことであり、効果検証手法の一つです。本文では、PythonでDIDを実装する方法について解説しています。また、DID以外にも、傾向スコアを用いた効果検証、操作変数を用いた効果検証、回帰不連続デザインを用いた効果検証、機械学習を用いた効果検証についても触れられています。
DIDとは、差分の差分法 (Difference-in-Differences) のことであり、効果検証手法の一つです。本文では、PythonでDIDを実装する方法について解説しています。また、DID以外にも、傾向スコアを用いた効果検証、操作変数を用いた効果検証、回帰不連続デザインを用いた効果検証、機械学習を用いた効果検証についても触れられています。
DEBUG:llama_index.indices.response.response_builder:> Initial prompt template: コンテキストは以下です. 
---------------------
url: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0

url: https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0

 * [傾向スコアを用いた効果検証](https://zenn.dev/s1ok69oo/arti

DEBUG:llama_index.indices.response.response_builder:> Initial response: DIDとは、差分の差分法 (Difference-in-Differences) のことであり、効果検証手法の一つです。本文では、PythonでDIDを実装する方法について解説しています。また、DID以外にも、傾向スコアを用いた効果検証、操作変数を用いた効果検証、回帰不連続デザインを用いた効果検証、機械学習を用いた効果検証についても触れられています。
> Initial response: DIDとは、差分の差分法 (Difference-in-Differences) のことであり、効果検証手法の一つです。本文では、PythonでDIDを実装する方法について解説しています。また、DID以外にも、傾向スコアを用いた効果検証、操作変数を用いた効果検証、回帰不連続デザインを用いた効果検証、機械学習を用いた効果検証についても触れられています。
DEBUG:llama_index.indices.response.response_builder:> Refine context:  │   ├── output.tf
        │   └── variable.tf
...
> Refine context:  │   ├── output.tf
        │   └── variable.tf
...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066\\n\\u

api_version=None data='{"messages": [{"role": "user", "content": "\\u5143\\u306e\\u8cea\\u554f: DID\\u306b\\u3064\\u3044\\u3066\\u6559\\u3048\\u3066\\n\\u30aa\\u30ea\\u30b8\\u30ca\\u30eb\\u306e\\u56de\\u7b54: DID\\u3068\\u306f\\u3001\\u5dee\\u5206\\u306e\\u5dee\\u5206\\u6cd5 (Difference-in-Differences) \\u306e\\u3053\\u3068\\u3067\\u3042\\u308a\\u3001\\u52b9\\u679c\\u691c\\u8a3c\\u624b\\u6cd5\\u306e\\u4e00\\u3064\\u3067\\u3059\\u3002\\u672c\\u6587\\u3067\\u306f\\u3001Python\\u3067DID\\u3092\\u5b9f\\u88c5\\u3059\\u308b\\u65b9\\u6cd5\\u306b\\u3064\\u3044\\u3066\\u89e3\\u8aac\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002\\u307e\\u305f\\u3001DID\\u4ee5\\u5916\\u306b\\u3082\\u3001\\u50be\\u5411\\u30b9\\u30b3\\u30a2\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c\\u3001\\u64cd\\u4f5c\\u5909\\u6570\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c\\u3001\\u56de\\u5e30\\u4e0d\\u9023\\u7d9a\\u30c7\\u30b6\\u30a4\\u30f3\\u3092\\u7528\\u3044\\u305f\\u52b9\\u679c\\u691c\\u8a3c\\u3001\\u6a

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=11843 request_id=b088c4f362e236a1f93c98cc09b2eeaa response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=11843 request_id=b088c4f362e236a1f93c98cc09b2eeaa response_code=200
DEBUG:llama_index.llm_predictor.base:申し訳ありませんが、元の質問に関する回答と、提供されたコンテキストとの間には明確な関連性がありません。DIDに関する回答は、効果検証手法の一つであり、Pythonで実装する方法について解説しています。一方、提供されたコンテキストは、Terraformを使用したインフラストラクチャの構築に関するものです。もし、より具体的な質問があれば、お答えできるかもしれません。
申し訳ありませんが、元の質問に関する回答と、提供されたコンテキストとの間には明確な関連性がありません。DIDに関する回答は、効果検証手法の一つであり、Pythonで実装する方法について解説しています。一方、提供されたコンテキストは、Terraformを使用したインフラストラクチャの構築に関するものです。もし、より具体的な質問があれば、お答えできるかもしれません。
DEBUG:llama_index.indices.response.response_builder:> Refined prompt template: 元の質問: DIDについて教えて
オリジナルの回答: DIDとは、差分の差分法 (Difference-in-Differences) のことであり、効果検証手法の一つです。本文では、PythonでDIDを実装する方法について解説しています。また、DID以外にも、傾向スコアを用いた効果検証、操作変数を用いた効果検証、回帰不連続デザインを用いた効果検証、機械学習を用いた

DEBUG:llama_index.indices.response.response_builder:> Refined response: 申し訳ありませんが、元の質問に関する回答と、提供されたコンテキストとの間には明確な関連性がありません。DIDに関する回答は、効果検証手法の一つであり、Pythonで実装する方法について解説しています。一方、提供されたコンテキストは、Terraformを使用したインフラストラクチャの構築に関するものです。もし、より具体的な質問があれば、お答えできるかもしれません。
> Refined response: 申し訳ありませんが、元の質問に関する回答と、提供されたコンテキストとの間には明確な関連性がありません。DIDに関する回答は、効果検証手法の一つであり、Pythonで実装する方法について解説しています。一方、提供されたコンテキストは、Terraformを使用したインフラストラクチャの構築に関するものです。もし、より具体的な質問があれば、お答えできるかもしれません。
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 6983 tokens
> [get_response] Total LLM token usage: 6983 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens
response: 申し訳ありませんが、元の質問に関する回答と、提供されたコンテキストとの間には明確な関連性がありません。DIDに関する回答は、効果検証手法の一つであり、Pythonで実装する方法について解説しています。一方、提供されたコンテキストは、Terraformを使用したインフラストラクチャの構築に関するものです。もし、より具体的な質問があれば、お答えできるかもしれません。
参照url1: https://qiita.com/nokoxxx1212/items/5

In [27]:
query_response = pinecone_index.query(
    top_k=10,
    include_metadata=True,
    # dummyのベクトル
    vector=[0.1] * 1536,
    filter={
         "extra_info_url": {"$in": ["https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0"]}
    }
)

In [32]:
len(query_response["matches"])

10

In [31]:
search_query = "Qdrantの特徴は?"

response = index.query(
    search_query,
    similarity_top_k=3,
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 15872 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 10 tokens


response: Qdrantの特徴は、オープンソースのRust製ベクトル検索エンジンであり、Python SDK、REST API、gRPCで接続できること、クラウドサービス版も準備中であること、ベクトル検索においてコサイン類似度を使用していること、他のベクトル検索エンジンと比較して高速であることなどが挙げられます。Qdrantは、機械学習関連のブログでよく使用されるlivedoorニュースコーパスなどのベクトルデータを取り込むことができます。また、Qdrantの他にも、Facebook FaissやPynndescentなどのライブラリもベクトル検索に使用できます。Qdrantは、コレクションとポイントの概念を持ち、コレクションはRDBのテーブルに、ポイントはRDBのレコードに相当します。Qdrantでは、ペイロードと呼ばれるメタ情報も一緒に登録でき、フィルター検索に使用できます。Python SDKを使用することで、コレクションの作成、ドキュメントの登録、類似ドキュメントの検索が可能です。また、Qdrantはベクトル検索した後でフィルタリングすることができるため、検索結果が少なくなることを防ぐことができます。さらに、Qdrantはバッチクエリーという機能を持ち、一度に複数のクエリーを投げることができます。また、レコメンドAPIを使用することで、インデックスされたポイントを使用して検索できます。
参照url1: https://zenn.dev/tfutada/articles/acf8adbb2ba5be


In [32]:
# indexをローカルに保存
index.save_to_disk("../data/pinecone_index.json")

# index insertエラー時

In [7]:
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENVIRONMENT"])
pinecone_index = pinecone.Index(INDEX_NAME)

In [9]:
llm_predictor = LLMPredictor(
    llm=OpenAI(
        temperature=0, model_name=PREDICTOR_MODEL_NAME, openai_api_key=os.environ["OPENAI_API_KEY"]
    )
)

embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL_NAME, openai_api_key=os.environ["OPENAI_API_KEY"]
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)

index = GPTPineconeIndex.from_documents(
    pinecone_index=pinecone_index,
    documents=[], service_context=service_context
)

TypeError: __init__() got an unexpected keyword argument 'openai_api_key'

In [16]:
additional_urls = ["https://note.com/npaka/n/n2e2cf5a458ac"]
additional_documents = SimpleWebPageReader(html_to_text=True).load_data(additional_urls)

# extra_infoにurlを追加
for additional_document, url in zip(additional_documents, additional_urls):
    additional_document.extra_info = {"url": url}
    index.insert(additional_document)

ValueError: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


In [6]:
from llama_index import OpenAIEmbeddingModelType

ImportError: cannot import name 'OpenAIEmbeddingModelType' from 'llama_index' (/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/llama_index/__init__.py)